# Separador de sentenças do Eproc SC

In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2
import os
import re

## Configuração dos Diretórios de Entrada e Saída

* O diretório de entrada é o que possui os arquivos PDFs com várias sentenças, e apenas isso
* O diretório de saída será onde as sentenças separadas serão armazenadas, um PDF por sentença

In [ ]:
input_folder = "input"

input_files = os.listdir(input_folder)
print(f'Todos arquivos no diretório de entrada: {input_files}')

In [ ]:
# Remove qualquer arquivo que não seja PDF da lista
for f in input_files:
    if not f.endswith('.pdf'):
        input_files.remove(f)

print(f'PDFs do diretório de entrada: {input_files}')

In [ ]:
# Diretório de saída, onde as sentenças separadas serão armazenadas
output_folder = "output"

# Verificar se o diretório não existe
if not os.path.exists(output_folder):
    # Criar o diretório de saída se não existir
    os.makedirs(output_folder)
    print(f"Diretório de saída: `{output_folder}` criado com sucesso!")
else:
    # Se existir, só avisa que já existe
    print(f"O diretório de saída `{output_folder}` já existe!")


In [ ]:
# Função para imprimir o texto na cor vermelha
def red(text):
    return f"\033[91m{text}\033[0m"

## Processamento de um arquivo de entrada

In [ ]:
# Função que lê um arquivo PDF (pdf_reader já aberto) e divide em múltiplos PDFs
# de acordo com a lista de páginas (pages) marcadas.
# O nome dos arquivos gerados são os números dos processos (sentence_ids)

def split_sentence_files(pdf_reader,pages,sentence_ids):
    # Marcador da primeira página 
    start_page = 0
    for doc_index, end_page in enumerate(pages):
        sentence_id = sentence_ids[doc_index]
        output_filename = f"{output_folder}/{sentence_id}.pdf"
        counter = 1
        while os.path.exists(output_filename): # Se o arquivo já existe, ou seja, uma sentença com o mesmo número de processo, gera um nome seguido de um número
            print(red(f'O arquivo `{output_filename}` já existe'))
            output_filename = f"{output_folder}/{sentence_id}({counter}).pdf"
            print(f'Criando o arquivo: {output_filename}')
            counter += 1
        
        
        print('- Em processamento: [',doc_index+1,"/",len(pages),"] : [", start_page, ",", end_page , '] - ', sentence_ids[doc_index], ":",output_filename)
        # Escreve arquivos na saída
        pdf_writer = PyPDF2.PdfWriter()
        pdf_writer.append(fileobj=pdf_reader, pages=(start_page, end_page))
        output = open(output_filename, "wb")
        pdf_writer.write(output)
        pdf_writer.close()
        output.close()
        
        start_page = end_page

## Processamento em lote dos arquivos de entrada

Para cada arquivo de entrada, realiza a operação de divisão

In [ ]:
# Processamento em lote de todos os arquivos no diretório de entrada!
# Para cada arquivo no diretório de entrada, divida-os em múltiplos no diretório de saída!

sentence_set = set() # Conjunto para identificar se alguma sentença repete!
sentence_ids_files = dict()

for file in input_files:
    filename = f'{input_folder}/{file}'
    print(f'>>>>> Processando arquivo: `{filename}`. AGUARDE...')
    # Ler arquivo de entrada
    pdf_fileobj = open(filename,'rb')
    
    # Criar o leitor do arquivo de entrada
    pdf_reader = PyPDF2.PdfReader(pdf_fileobj)
    
    # Lista com as marcações das páginas de início e final de sentença
    pages_divisors = [] # Vetor com os números das páginas que iniciam cada sentença
    sentence_ids = []   # Vetor com o Identificador da sentença que é o número do processo
    for i in range(len(pdf_reader.pages)): # Para cada página do arquivo aberto
        page_obj=pdf_reader.pages[i]       # Lê a página
        page_text=page_obj.extract_text()  # Extrai o texto
        page_lines = page_text.split('\n') # Quebra as linhas
        page_first_line = page_lines[0]    # Obtém o texto da primeira linha

        # O que identifica a primeira página de cada sentença é o endereço da UFSC (baseado no arquivo de teste)
        if page_first_line == 'Av. Des. Vitor Lima, 183, fundos- Campus da UFSC - Bairro: Serrinha - CEP: 88040-400 - Fone: (48)3287-5019 - Email:':
            sentence_id = None 
            for line_index in range(-6,-1): # Busca identificador da sentença nas últimas 6 linhas do texto.
                sentence_id = re.match('[0-9]+-[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+',page_lines[line_index])
                if sentence_id is not None: # Quando encontrar, para!
                    break

            if sentence_id.group() in sentence_set:
                for sentence_file in sentence_ids_files:
                    if sentence_id.group() in sentence_ids_files[sentence_file]:
                        print(red(f'A sentença {sentence_id.group()} já está no conjunto, pois apareceu no arquivo: {sentence_file}!'))
                
            sentence_set.add(sentence_id.group())
            pages_divisors.append(i)                  # Marca a página de início da sentença
            sentence_ids.append(sentence_id.group())  # O identificador da sentença (número do processo) que está no rodapé da página é registrado

    sentence_ids_files[f'{file}'] = sentence_ids
    pages_divisors.pop(0) # Remove o marcador de página 0, pois página que inícia a primeira sentença sempre será a primeira
    pages_divisors.append(len(pdf_reader.pages)) # Adiciona a última página do arquivo na lista de páginas.

    print(f'## Páginas que dividem as sentenças no arquivo `{file}`: {pages_divisors}')
    print(f'## Número dos processos do arquivo `{file}`: {sentence_ids}')
    split_sentence_files(pdf_reader,pages_divisors,sentence_ids) # Processa o arquivo
    pdf_fileobj.close()
    print()

print(red("\n\n######## CONCLUÍDO ########"))

In [ ]:
print('Sentenças encontradas em cada arquivo!')
sentence_ids_files

# Extrator de Texto

Lê os arquivos gerados no processamento em lote da etapa anterior, extrai o texto dos arquivos e gera um arquvo TXT apenas com o texto útil dos arquivos.

O texto útil é resultado da:
1. Remoção dos cabeçalhos
2. 

In [ ]:
# Usando o diretório de saída do processamento anterior,
# pois é nele que estão todos os PDFs das sentenças individualizadas
# para a extração de texto.
input_folder = "output"  

print('Arquivos para leitura no diretório de entrada:')
input_files = os.listdir(input_folder)
input_files